In [150]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew #加载一些统计的库
import time
import gc
import math

In [151]:
print('train')
train = pd.read_table('train.txt',sep=' ',index_col=False)

train


In [152]:
train.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade'],
      dtype='object')

In [153]:
train = train[['instance_id','item_id','is_trade', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description','context_page_id']]

In [154]:
print('test')
test = pd.read_table('test.txt',sep=' ',index_col=False)

test


In [155]:
test = test[['instance_id','item_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description','context_page_id']]

In [156]:
train[['item_id','is_trade']].groupby('item_id').sum()/train[['item_id','is_trade']].groupby('item_id').count()

,is_trade
item_id,
42172734419742,0.018416
119705681289325,0.000000
139232509090699,0.000000
186228696000288,0.000000
208192717221960,0.000000
271501162750665,0.012384
334417922587028,0.000000
345352588519597,0.022388
412546248320450,0.000000


In [157]:
temp = pd.DataFrame()

In [158]:
temp['item_id'] = (train[['item_id','is_trade']].groupby('item_id').sum()/train[['item_id','is_trade']].groupby('item_id').count()).reset_index()[['item_id']]

In [159]:
temp['pv'] = (train[['item_id','is_trade']].groupby('item_id').sum()/train[['item_id','is_trade']].groupby('item_id').count()).reset_index()[['is_trade']]

In [160]:
all_data = train.append(test)

In [161]:
all_data

,context_page_id,instance_id,is_trade,item_collected_level,item_id,item_price_level,item_pv_level,item_sales_level,shop_review_num_level,shop_review_positive_rate,shop_score_delivery,shop_score_description,shop_score_service,shop_star_level
0,4007,7118167540371,0.0,12,7595487976415452415,8,14,10,11,0.988447,0.971962,0.972170,0.974737,5009
1,4001,43508800019098,0.0,15,5340996313837249587,8,20,15,17,1.000000,0.959580,0.979941,0.963293,5015
2,4004,78067858668749,0.0,13,8363638779510498577,8,20,10,18,1.000000,0.983493,0.987826,0.983985,5015
3,4001,137525720365644,0.0,10,7121266959776152715,7,17,10,17,1.000000,0.964313,0.978024,0.964843,5014
4,4001,151193932520734,0.0,14,5738516817201102842,7,18,11,16,1.000000,0.958688,0.978506,0.960387,5013
5,4001,191034067765499,0.0,11,3165167092384011466,7,14,9,13,0.995213,0.977785,0.975474,0.977124,5011
6,4001,221786632908010,0.0,13,2157590763338091652,5,18,14,15,0.989038,0.955115,0.942377,0.952626,5012
7,4001,233375766178961,0.0,11,7296864533820220362,8,17,9,16,1.000000,0.987170,0.989783,0.986273,5013
8,4004,235542268576836,0.0,16,3948722777398180782,8,20,11,18,1.000000,0.969538,0.982733,0.969531,5015
9,4018,271028956022181,0.0,11,4158769038163595899,7,16,9,17,1.000000,0.962803,0.974162,0.963310,5014


In [162]:
all_data = pd.merge(all_data,temp,on=['item_id'],how='left')

In [163]:
all_data.count()

context_page_id              10951924
instance_id                  10951924
is_trade                     10432036
item_collected_level         10951924
item_id                      10951924
item_price_level             10951924
item_pv_level                10951924
item_sales_level             10951924
shop_review_num_level        10951924
shop_review_positive_rate    10951924
shop_score_delivery          10951924
shop_score_description       10951924
shop_score_service           10951924
shop_star_level              10951924
pv                           10949047
dtype: int64

In [164]:
temp = pd.DataFrame()

In [165]:
temp['item_id'] = all_data[['item_id','instance_id']].groupby('item_id').count().reset_index()[['item_id']]

In [166]:
temp['count'] = all_data[['item_id','instance_id']].groupby('item_id').count().reset_index()[['instance_id']]

In [167]:
temp

,item_id,count
0,29268323426272,1
1,42172734419742,619
2,119705681289325,2
3,139232509090699,79
4,186228696000288,3
5,208192717221960,5
6,271501162750665,341
7,334417922587028,2
8,345352588519597,137
9,412546248320450,1


In [168]:
all_data = pd.merge(all_data,temp,on=['item_id'],how='left')

In [169]:
all_data

,context_page_id,instance_id,is_trade,item_collected_level,item_id,item_price_level,item_pv_level,item_sales_level,shop_review_num_level,shop_review_positive_rate,shop_score_delivery,shop_score_description,shop_score_service,shop_star_level,pv,count
0,4007,7118167540371,0.0,12,7595487976415452415,8,14,10,11,0.988447,0.971962,0.972170,0.974737,5009,0.011050,181
1,4001,43508800019098,0.0,15,5340996313837249587,8,20,15,17,1.000000,0.959580,0.979941,0.963293,5015,0.027311,8529
2,4004,78067858668749,0.0,13,8363638779510498577,8,20,10,18,1.000000,0.983493,0.987826,0.983985,5015,0.001588,11869
3,4001,137525720365644,0.0,10,7121266959776152715,7,17,10,17,1.000000,0.964313,0.978024,0.964843,5014,0.032340,1175
4,4001,151193932520734,0.0,14,5738516817201102842,7,18,11,16,1.000000,0.958688,0.978506,0.960387,5013,0.010422,2293
5,4001,191034067765499,0.0,11,3165167092384011466,7,14,9,13,0.995213,0.977785,0.975474,0.977124,5011,0.000000,94
6,4001,221786632908010,0.0,13,2157590763338091652,5,18,14,15,0.989038,0.955115,0.942377,0.952626,5012,0.057519,1776
7,4001,233375766178961,0.0,11,7296864533820220362,8,17,9,16,1.000000,0.987170,0.989783,0.986273,5013,0.001535,2028
8,4004,235542268576836,0.0,16,3948722777398180782,8,20,11,18,1.000000,0.969538,0.982733,0.969531,5015,0.004620,9524
9,4018,271028956022181,0.0,11,4158769038163595899,7,16,9,17,1.000000,0.962803,0.974162,0.963310,5014,0.000000,421


In [170]:
all_data.count()

context_page_id              10951924
instance_id                  10951924
is_trade                     10432036
item_collected_level         10951924
item_id                      10951924
item_price_level             10951924
item_pv_level                10951924
item_sales_level             10951924
shop_review_num_level        10951924
shop_review_positive_rate    10951924
shop_score_delivery          10951924
shop_score_description       10951924
shop_score_service           10951924
shop_star_level              10951924
pv                           10949047
count                        10951924
dtype: int64

In [171]:
all_data = all_data[['item_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description','context_page_id','pv','count']].groupby('item_id').mean().reset_index()

In [172]:
all_data = all_data.fillna(-1)

In [173]:
all_data.shape

(86461, 14)

In [174]:
all_data.drop_duplicates(subset='item_id').shape

(86461, 14)

In [175]:
all_data.count()

item_id                      86461
item_price_level             86461
item_sales_level             86461
item_collected_level         86461
item_pv_level                86461
shop_review_num_level        86461
shop_review_positive_rate    86461
shop_star_level              86461
shop_score_service           86461
shop_score_delivery          86461
shop_score_description       86461
context_page_id              86461
pv                           86461
count                        86461
dtype: int64

In [176]:
all_data

,item_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,context_page_id,pv,count
0,29268323426272,5.0,4.000000,5.000000,9.000000,11.000000,0.976212,5009.000000,0.988571,0.988571,0.980952,4001.000000,-1.000000,1.0
1,42172734419742,7.0,12.000000,13.000000,17.347334,18.000000,1.000000,5015.000000,0.970964,0.969183,0.982560,4003.838449,0.018416,619.0
2,119705681289325,7.0,4.000000,4.000000,10.000000,5.500000,0.992647,5003.000000,0.988966,0.988966,0.983448,4006.000000,0.000000,2.0
3,139232509090699,7.0,4.000000,7.000000,12.075949,17.000000,0.986221,5015.000000,0.976953,0.976082,0.969057,4005.063291,0.000000,79.0
4,186228696000288,7.0,3.000000,5.000000,12.000000,15.000000,0.962455,5012.000000,1.000000,1.000000,1.000000,4002.666667,0.000000,3.0
5,208192717221960,5.0,8.000000,10.000000,11.600000,18.000000,1.000000,5015.000000,0.974683,0.975921,0.978280,4001.800000,0.000000,5.0
6,271501162750665,6.0,11.000000,11.000000,16.000000,16.000000,0.995533,5014.000000,0.972537,0.971900,0.968268,4003.551320,0.012384,341.0
7,334417922587028,6.0,6.000000,10.000000,6.000000,12.000000,0.984570,5009.000000,0.980260,0.980195,0.975195,4001.000000,0.000000,2.0
8,345352588519597,5.0,9.000000,10.000000,14.000000,13.000000,0.974718,5011.000000,0.966927,0.969220,0.962784,4003.080292,0.022388,137.0
9,412546248320450,7.0,3.000000,6.000000,9.000000,18.000000,0.997523,5015.000000,0.973754,0.974400,0.966921,4001.000000,0.000000,1.0


In [177]:
test_data1 = all_data[all_data['pv'] == -1]

In [186]:
all_data = all_data[all_data['pv'] != -1]

In [178]:
test_data1

,item_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,context_page_id,pv,count
0,29268323426272,5.0,4.0,5.0,9.0,11.0,0.976212,5009.0,0.988571,0.988571,0.980952,4001.000000,-1.0,1.0
64,6474333064915340,5.0,2.0,9.0,9.0,23.0,0.998533,5020.0,0.980368,0.979966,0.972939,4001.000000,-1.0,1.0
85,7902290000120447,4.0,6.0,5.0,7.0,13.0,0.988504,5011.0,0.963373,0.962224,0.956006,4004.000000,-1.0,1.0
173,16922360910898786,7.0,6.0,7.0,6.0,19.0,0.996264,5016.0,0.969971,0.969399,0.964373,4009.000000,-1.0,1.0
187,17745429455414436,7.0,6.0,9.0,8.0,18.0,1.000000,5015.0,0.981455,0.981139,0.989066,4019.000000,-1.0,1.0
224,21665219455557253,8.0,1.0,5.0,9.0,9.0,0.978764,5007.0,0.950130,0.958442,0.957403,4003.000000,-1.0,1.0
236,23148230178539378,7.0,5.0,10.0,5.0,18.0,1.000000,5015.0,0.977895,0.977141,0.988600,4010.000000,-1.0,1.0
337,33424355669209869,8.0,2.0,5.0,9.0,14.0,1.000000,5012.0,0.971877,0.973584,0.982222,4001.000000,-1.0,1.0
357,36271664435921368,8.0,6.0,7.0,9.0,14.0,0.998243,5012.0,0.978343,0.984162,0.974788,4001.000000,-1.0,1.0
390,39461555088536940,4.0,1.0,5.0,8.0,19.0,0.991437,5016.0,0.979038,0.979686,0.970129,4015.000000,-1.0,1.0


In [179]:
test_data2 = all_data[all_data['count'] < 50]

In [187]:
all_data = all_data[all_data['count'] >= 50]

In [180]:
test_data2

,item_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,context_page_id,pv,count
0,29268323426272,5.0,4.000000,5.000000,9.000000,11.0,0.976212,5009.0,0.988571,0.988571,0.980952,4001.000000,-1.000000,1.0
2,119705681289325,7.0,4.000000,4.000000,10.000000,5.5,0.992647,5003.0,0.988966,0.988966,0.983448,4006.000000,0.000000,2.0
4,186228696000288,7.0,3.000000,5.000000,12.000000,15.0,0.962455,5012.0,1.000000,1.000000,1.000000,4002.666667,0.000000,3.0
5,208192717221960,5.0,8.000000,10.000000,11.600000,18.0,1.000000,5015.0,0.974683,0.975921,0.978280,4001.800000,0.000000,5.0
7,334417922587028,6.0,6.000000,10.000000,6.000000,12.0,0.984570,5009.0,0.980260,0.980195,0.975195,4001.000000,0.000000,2.0
9,412546248320450,7.0,3.000000,6.000000,9.000000,18.0,0.997523,5015.0,0.973754,0.974400,0.966921,4001.000000,0.000000,1.0
10,584480686684468,8.0,3.000000,5.000000,11.000000,9.0,0.973236,5007.0,0.944369,0.938462,0.917292,4007.666667,0.000000,3.0
11,797382621888539,8.0,3.000000,6.000000,7.000000,14.0,0.983319,5011.0,0.948412,0.951535,0.942424,4002.000000,0.000000,1.0
12,1026777098080416,8.0,4.000000,5.000000,4.000000,17.0,1.000000,5014.0,0.957740,0.958537,0.976138,4006.000000,0.000000,1.0
13,1189938696856115,4.0,3.375000,4.000000,13.000000,12.0,0.981048,5009.0,0.989985,0.988584,0.985782,4004.937500,0.000000,32.0


In [181]:
test_data = test_data1.append(test_data2)

In [182]:
test_data

,item_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,context_page_id,pv,count
0,29268323426272,5.0,4.000000,5.000000,9.000000,11.0,0.976212,5009.0,0.988571,0.988571,0.980952,4001.000000,-1.000000,1.0
64,6474333064915340,5.0,2.000000,9.000000,9.000000,23.0,0.998533,5020.0,0.980368,0.979966,0.972939,4001.000000,-1.000000,1.0
85,7902290000120447,4.0,6.000000,5.000000,7.000000,13.0,0.988504,5011.0,0.963373,0.962224,0.956006,4004.000000,-1.000000,1.0
173,16922360910898786,7.0,6.000000,7.000000,6.000000,19.0,0.996264,5016.0,0.969971,0.969399,0.964373,4009.000000,-1.000000,1.0
187,17745429455414436,7.0,6.000000,9.000000,8.000000,18.0,1.000000,5015.0,0.981455,0.981139,0.989066,4019.000000,-1.000000,1.0
224,21665219455557253,8.0,1.000000,5.000000,9.000000,9.0,0.978764,5007.0,0.950130,0.958442,0.957403,4003.000000,-1.000000,1.0
236,23148230178539378,7.0,5.000000,10.000000,5.000000,18.0,1.000000,5015.0,0.977895,0.977141,0.988600,4010.000000,-1.000000,1.0
337,33424355669209869,8.0,2.000000,5.000000,9.000000,14.0,1.000000,5012.0,0.971877,0.973584,0.982222,4001.000000,-1.000000,1.0
357,36271664435921368,8.0,6.000000,7.000000,9.000000,14.0,0.998243,5012.0,0.978343,0.984162,0.974788,4001.000000,-1.000000,1.0
390,39461555088536940,4.0,1.000000,5.000000,8.000000,19.0,0.991437,5016.0,0.979038,0.979686,0.970129,4015.000000,-1.000000,1.0


In [185]:
test_data = test_data.drop_duplicates(subset='item_id')

In [184]:
import lightgbm as lgb

In [233]:
all_data

,item_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,context_page_id,pv,count
1,42172734419742,7.000000,12.000000,13.000000,17.347334,18.000000,1.000000,5015.000000,0.970964,0.969183,0.982560,4003.838449,0.018416,619.0
3,139232509090699,7.000000,4.000000,7.000000,12.075949,17.000000,0.986221,5015.000000,0.976953,0.976082,0.969057,4005.063291,0.000000,79.0
6,271501162750665,6.000000,11.000000,11.000000,16.000000,16.000000,0.995533,5014.000000,0.972537,0.971900,0.968268,4003.551320,0.012384,341.0
8,345352588519597,5.000000,9.000000,10.000000,14.000000,13.000000,0.974718,5011.000000,0.966927,0.969220,0.962784,4003.080292,0.022388,137.0
15,1410182414862995,6.000000,7.000000,9.468660,15.815815,18.000000,0.987696,5015.000000,0.969681,0.973725,0.963582,4004.647059,0.001021,1037.0
18,1629778605881953,7.000000,5.420455,5.818182,13.420455,14.000000,0.987025,5011.000000,0.965697,0.966775,0.958570,4005.045455,0.015873,88.0
21,2135374266088353,5.000000,15.000000,14.000000,17.000000,18.000000,0.986122,5015.000000,0.975087,0.974328,0.970459,4002.216710,0.048518,766.0
23,2303426035633108,6.000000,7.818182,7.672727,11.672727,17.000000,0.992026,5014.000000,0.971393,0.970173,0.963863,4004.381818,0.000000,55.0
25,2530623823134608,6.000000,7.527027,9.000000,12.175676,17.527027,1.000000,5015.000000,0.976293,0.975201,0.982045,4004.081081,0.013889,74.0
33,3537019921042620,10.000000,6.140351,10.000000,14.890351,19.000000,1.000000,5016.000000,0.987598,0.988129,0.991467,4002.570175,0.004484,228.0


In [188]:
all_data.columns

Index(['item_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'shop_review_num_level',
       'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
       'shop_score_delivery', 'shop_score_description', 'context_page_id',
       'pv', 'count'],
      dtype='object')

In [215]:
X_train = np.array(all_data[[ 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'shop_review_num_level',
       'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
       'shop_score_delivery', 'shop_score_description', 'context_page_id']].copy())

In [216]:
y_train = np.array(all_data[[ 'pv']].copy())

In [217]:
X_test = np.array(test_data[[ 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'shop_review_num_level',
       'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
       'shop_score_delivery', 'shop_score_description', 'context_page_id']].copy())

In [218]:
lgb_train= lgb.Dataset(X_train,y_train)

In [ ]:
# lgb_eval = lgb.Datasetaset()

In [197]:
params = {
    'task':'train',
    'boosting_type':'gbdt',
    'numleaves':64,
    'max_depth':8,
    'learning_rate':0.01,
    'n_estimators':2000,
    'colsample_bytree':0.65,
    'subsample':0.7,
    'reg_alpha':2
}

In [223]:
gbm = lgb.LGBMRegressor(
                        num_leaves = 128,
#                         max_depth = 8,
                        device = "gpu",
                        gpu_platform_id = 0,
                        gpu_device_id = 0,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 1,)

In [224]:
X_train

array([[  7.00000000e+00,   1.20000000e+01,   1.30000000e+01, ...,
          9.69182679e-01,   9.82560069e-01,   4.00383845e+03],
       [  7.00000000e+00,   4.00000000e+00,   7.00000000e+00, ...,
          9.76081812e-01,   9.69057124e-01,   4.00506329e+03],
       [  6.00000000e+00,   1.10000000e+01,   1.10000000e+01, ...,
          9.71899955e-01,   9.68267796e-01,   4.00355132e+03],
       ..., 
       [  1.00000000e+00,   1.40000000e+01,   1.50000000e+01, ...,
          9.74479573e-01,   9.67300956e-01,   4.00209172e+03],
       [  6.00000000e+00,   1.11174377e+01,   1.19430605e+01, ...,
          9.62398045e-01,   9.73299245e-01,   4.00281673e+03],
       [  6.00000000e+00,   5.05813953e+00,   5.94186047e+00, ...,
          9.75822461e-01,   9.69769968e-01,   4.00306977e+03]])

In [226]:
gbm.fit(X_train,y_train)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       device='gpu', gpu_device_id=0, gpu_platform_id=0,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=128, objective=None, random_state=None,
       reg_alpha=1, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [227]:
y_test = gbm.predict(X_test)

In [228]:
y_test

array([ 0.01025611,  0.00839923,  0.020313  , ...,  0.00638489,
        0.00898035,  0.01002099])

In [229]:
temp = pd.DataFrame()

In [240]:
temp['item_id'] = all_data[[ 'item_id']].copy().reset_index()[['item_id']]

In [241]:
temp

,item_id
0,42172734419742
1,139232509090699
2,271501162750665
3,345352588519597
4,1410182414862995
5,1629778605881953
6,2135374266088353
7,2303426035633108
8,2530623823134608
9,3537019921042620


In [242]:
temp1 = pd.DataFrame(y_train)

In [243]:
temp1

,0
0,0.018416
1,0.000000
2,0.012384
3,0.022388
4,0.001021
5,0.015873
6,0.048518
7,0.000000
8,0.013889
9,0.004484


In [246]:
temp['pv'] = temp1[[0]].copy().reset_index()[[0]]

In [247]:
temp

,item_id,pv
0,42172734419742,0.018416
1,139232509090699,0.000000
2,271501162750665,0.012384
3,345352588519597,0.022388
4,1410182414862995,0.001021
5,1629778605881953,0.015873
6,2135374266088353,0.048518
7,2303426035633108,0.000000
8,2530623823134608,0.013889
9,3537019921042620,0.004484


In [249]:
temp2 = temp.copy()

In [250]:
temp = pd.DataFrame()
temp['item_id'] = test_data[[ 'item_id']].copy().reset_index()[['item_id']]
temp1 = pd.DataFrame(y_test)
temp['pv'] = temp1[[0]].copy().reset_index()[[0]]

In [251]:
temp

,item_id,pv
0,29268323426272,0.010256
1,6474333064915340,0.008399
2,7902290000120447,0.020313
3,16922360910898786,0.007694
4,17745429455414436,0.006354
5,21665219455557253,0.008908
6,23148230178539378,0.005697
7,33424355669209869,0.004989
8,36271664435921368,0.008099
9,39461555088536940,0.008137


In [252]:
temp2 = temp2.append(temp)

In [253]:
temp2

,item_id,pv
0,42172734419742,0.018416
1,139232509090699,0.000000
2,271501162750665,0.012384
3,345352588519597,0.022388
4,1410182414862995,0.001021
5,1629778605881953,0.015873
6,2135374266088353,0.048518
7,2303426035633108,0.000000
8,2530623823134608,0.013889
9,3537019921042620,0.004484


In [254]:
data = pd.read_table('train.txt',sep=' ',index_col=False)
data2 = pd.read_table('test.txt',sep=' ',index_col=False)

In [255]:
data = pd.merge(data[['instance_id','item_id']],temp2[['item_id','pv']],on=['item_id'],how='left')
data2 = pd.merge(data2[['instance_id','item_id']],temp2[['item_id','pv']],on=['item_id'],how='left')

In [256]:
data

,instance_id,item_id,pv
0,7118167540371,7595487976415452415,0.011050
1,43508800019098,5340996313837249587,0.027311
2,78067858668749,8363638779510498577,0.001588
3,137525720365644,7121266959776152715,0.032340
4,151193932520734,5738516817201102842,0.010422
5,191034067765499,3165167092384011466,0.000000
6,221786632908010,2157590763338091652,0.057519
7,233375766178961,7296864533820220362,0.001535
8,235542268576836,3948722777398180782,0.004620
9,271028956022181,4158769038163595899,0.000000


In [257]:
data2

,instance_id,item_id,pv
0,93294255633855,2615021910672727866,0.011546
1,558322259509454,3748469724871724205,0.000401
2,594573634113186,2111357467597282780,0.044444
3,667327653735176,3717651022681022321,0.013274
4,697732672924394,8284923347262844848,0.004016
5,1467522731866334,122990815260714082,0.026846
6,2564345731136577,5947872684354624624,0.020698
7,2635932702100660,4376223620063302249,0.002506
8,3208825666465730,5226910727221958616,0.029990
9,3654424870386483,1981860414878829638,0.020518


In [258]:
data[['instance_id','pv']].to_csv('train_pv.csv',index=False) 

In [259]:
data2[['instance_id','pv']].to_csv('test_pv.csv',index=False) 